# Jaeger-query with trace DSL

This notebook loads a trace from Jaeger query service and runs analysis on it.

## Install library to the local maven repository
This step is only needed if trace DSL source code has been modified.
Open terminal in Jupyter and run the following command:
```
cd work && ./mvnw clean install -DskipTests
```

In [1]:
%%loadFromPOM
<dependency>
  <groupId>com.google.protobuf</groupId>
  <artifactId>protobuf-java</artifactId>
  <version>3.11.4</version>
</dependency>
<dependency>
  <groupId>com.google.protobuf</groupId>
  <artifactId>protobuf-java-util</artifactId>
  <version>3.11.4</version>
</dependency>
<dependency>
  <groupId>io.grpc</groupId>
  <artifactId>grpc-protobuf</artifactId>
  <version>1.28.0</version>
</dependency>
<dependency>
  <groupId>io.grpc</groupId>
  <artifactId>grpc-netty</artifactId>
  <version>1.28.0</version>
</dependency>
<dependency>
  <groupId>io.grpc</groupId>
  <artifactId>grpc-stub</artifactId>
  <version>1.28.0</version>
</dependency>

<dependency>
    <groupId>io.prometheus</groupId>
    <artifactId>simpleclient_httpserver</artifactId>
    <version>0.7.0</version>
</dependency>
<dependency>
    <groupId>org.apache.tinkerpop</groupId>
    <artifactId>tinkergraph-gremlin</artifactId>
    <version>3.4.3</version>
</dependency>
<dependency>
    <groupId>log4j</groupId>
    <artifactId>log4j</artifactId>
    <version>1.2.17</version>
</dependency>


In [2]:
%maven io.jaegertracing:jaeger-tracedsl:0.2.1-SNAPSHOT
%maven io.jaegertracing:jaeger-proto:0.2.1-SNAPSHOT

System.out.println(io.jaegertracing.analytics.gremlin.Keys.class);

class io.jaegertracing.analytics.gremlin.Keys


## Run analysis
It will display maximum height of the trace

In [7]:
import org.apache.tinkerpop.gremlin.structure.Graph;
import org.apache.tinkerpop.gremlin.structure.Vertex;
import io.jaegertracing.analytics.gremlin.*;
import io.jaegertracing.analytics.model.*;
import io.jaegertracing.analytics.*;
import io.grpc.ManagedChannel;
import io.grpc.ManagedChannelBuilder;
import io.jaegertracing.api_v2.Query.*;
import java.math.BigInteger;
import com.google.protobuf.ByteString;
import io.jaegertracing.api_v2.QueryServiceGrpc.QueryServiceBlockingStub;
import io.jaegertracing.api_v2.QueryServiceGrpc;

String queryHostPort = "192.168.0.150:16686";
String traceId = "28aac91d30f1512b";

ManagedChannel channel = ManagedChannelBuilder.forTarget(queryHostPort).usePlaintext().build();
QueryServiceBlockingStub queryService = QueryServiceGrpc.newBlockingStub(channel);
GetOperationsResponse operations = queryService
        .getOperations(GetOperationsRequest.newBuilder().setService("jaeger-query").build());
System.out.println(operations);

ByteString traceId = ByteString.copyFrom(new BigInteger(traceId, 16).toByteArray());
Iterator<SpansResponseChunk> traceProto = queryService.getTrace(GetTraceRequest.newBuilder().setTraceId(traceId).build());

Trace trace = Converter.toModel(traceProto.next().getSpansList());
System.out.println(trace.traceId);

Graph graph = GraphCreator.create(trace);
int height = TraceHeight.calculate(graph);
System.out.println(height);

operationNames: "/api/traces"
operationNames: "/api/services"
operationNames: "/api/traces/{traceID}"
operationNames: "/api/services/{service}/operations"
operations {
  name: "/api/services/{service}/operations"
  span_kind: "server"
}
operations {
  name: "/api/traces"
  span_kind: "server"
}
operations {
  name: "/api/services"
  span_kind: "server"
}
operations {
  name: "/api/traces/{traceID}"
  span_kind: "server"
}



CompilationException: 

## Write your own data analysis with trace DSL

In [10]:
TraceTraversal<Vertex, Vertex> traversal = graph.traversal(TraceTraversalSource.class)
        .hasName("/api/traces/{traceID}")
        .repeat(__.out())
        .until(__.hasName("multiRead"));

traversal.forEachRemaining(v -> GraphDSLExamples.printVertex(v));


multiRead[3af34324f870e57e:53e1f3ef200f8a1], tags = [traceId, spanId, duration, internal.span.format, startTime, operationName, parentId]
